In [12]:
import numpy as np
import cv2
from tkinter import *
import tkinter as tk
from tkinter import filedialog
window = tk.Tk()
window.geometry("500x300")
window.configure(background='#10cfc9')
def HSV_color_segmentation( f, H1, H2, S1, S2, V1, V2 ):
	g = f.copy( )
	nr, nc = f.shape[:2]
	hsv = cv2.cvtColor( f, cv2.COLOR_BGR2HSV )
	for x in range( nr ):
		for y in range( nc ):
			H = hsv[x,y,0] * 2
			S = hsv[x,y,1] / 255 * 100
			V = hsv[x,y,2] / 255 * 100
			if not( H >= H1 and H <= H2 and S >= S1 and S <= S2
				     and V >= V1 and V <= V2 ):
				g[x,y,0] = g[x,y,1] = g[x,y,2] = 0
	return g
def overlay_transparent(background, overlay, x, y):

    background_width = background.shape[1]
    background_height = background.shape[0]

    if x >= background_width or y >= background_height:
        return background

    h, w = overlay.shape[0], overlay.shape[1]

    if x + w > background_width:
        w = background_width - x
        overlay = overlay[:, :w]

    if y + h > background_height:
        h = background_height - y
        overlay = overlay[:h]

    if overlay.shape[2] < 4:
        overlay = np.concatenate(
            [
                overlay,
                np.ones((overlay.shape[0], overlay.shape[1], 1), dtype = overlay.dtype) * 255
            ],
            axis = 2,
        )

    overlay_image = overlay[..., :3]
    mask = overlay[..., 3:] / 255.0

    background[y:y+h, x:x+w] = (1.0 - mask) * background[y:y+h, x:x+w] + mask * overlay_image

    return background
def main( ):
	img_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
	img1 = cv2.resize(cv2.imread(img_path), (480, 315),interpolation=cv2.INTER_AREA)
	overlay = HSV_color_segmentation( img1, 55, 65, 90, 100, 90, 100 )

	background_path = filedialog.askopenfilename(initialdir = "/",title = "Select file",filetypes = (("jpeg files","*.jpg"), ("png files","*.png"), ("gif files","*.gif"), ("all files","*.*")))
	background = cv2.resize(cv2.imread(background_path), (480, 315),interpolation=cv2.INTER_AREA)
	background = overlay_transparent(background, overlay, 50,50)
	cv2.imshow('img_add',background)
	cv2.imwrite('img_add.png',background)    
	cv2.waitKey( 0 )
    
button = tk.Button(window, text = "魚眼", width = 10, height = 2, command = main,bg='#c8c8c8',activeforeground='red')
button.place(x = 20, y = 50)

window.mainloop()